The top 2018 International Destinations by Euromonitor

I wanted to see what the top destinations are, where they're located, any other information I can find on the city like population, location, number of immigrants, number of tourist spots, country happiness index, etc.

Main dataset source: http://go.euromonitor.com/rs/805-KOK-719/images/wpTop100CitiesEN_Final.pdf?mkt_tok=eyJpIjoiTkRrMk56VmpaVEZoWXprMSIsInQiOiJBTE5LZFM1cm00c1huY0hDRmRjSm1VNTAxYXlNUEdYSHB3eVwveHFJNUw2N0lBNTVncjB2QUpwaGdCQ2VFN1hEaXVKNGg4cnkzQ1wvcmZqV1FcLzV5QmpLbnlLY1loNVQyTTF1ZUxDemY1eDFWZnFocFJjcWhwUWJGVEUwcTlvYjMzXC8ifQ%3D%3D

Python sources: https://pandas.pydata.org/pandas-docs/stable/getting_started/comparison/comparison_with_sql.html

Project created around August, 2019 by Amelia V. Final set is shared in Tableau Public.

In [4]:
import csv, re, pandas, requests, bs4, webbrowser, sys

STEP ONE- CREATE INITIAL SET
I copied the table with the 100 cities from the euromonitor pdf into a text file.
The rows are separated by new lines and columns are separated by spaces. However there are a few cities and countries that are more than one word (ie. New York City) so I manually added an underscore _ to those

In [6]:
#Read in the text file and save the contents as a string
file1 = open("initial_euromonitor_report.txt","r")
initialset= file1.read()
file1.close()

In [7]:
#Make data into something readable as a csv
initialset= initialset.replace(' ',';')  #make all the spaces into semicolons
initialset= initialset.replace(',','')   #get rid of commas
initialset= initialset.replace('_',' ')  #change _ back into a space
#print(initialset)  #check to make sure the data looks how you want

In [8]:
#Save the string as a csv
file2 = open("formatted_euromonitor_report.csv","w")
file2.write(initialset)
file2.close()

In [9]:
#Read in the csv as a dataframe
topcities = pandas.read_csv("formatted_euromonitor_report.csv", sep=";")
#topcities   #check to make sure the data looks how you want

In [10]:
#Check to see if it's a number data type. If not, get rid of the commas and make them floats
topcities.dtypes
#topcities['Arrivals 2016'] = topcities['Arrivals 2016'].str.replace(',', '').astype(float)
#topcities['Arrivals 2017'] = topcities['Arrivals 2017'].str.replace(',', '').astype(float)
#topcities['Arrivals 2018'] = topcities['Arrivals 2018'].str.replace(',', '').astype(float)
#According to the report, 3 columns are not their actual numerical values and need to be multiplied by 1000. Only do this once
topcities['Arrivals 2016']=topcities['Arrivals 2016']*1000
topcities['Arrivals 2017']=topcities['Arrivals 2017']*1000
topcities['Arrivals 2018']=topcities['Arrivals 2018']*1000
#topcities   #check to make sure the data looks how you want
#topcities.to_csv(r'topvisitedcities_transformed.csv')

STEP TWO- ADD COUNTRY CODES
Country code dataset found here: https://www.kaggle.com/juanumusic/countries-iso-codes

In [12]:
#Save it as a dataframe
countrycode= pandas.read_csv("wikipedia-iso-country-codes.csv")
#remove unneeded columns and rename columns to Country and country code
countrycode= countrycode.drop(['Alpha-3 code','Numeric code','ISO 3166-2'], axis=1)
countrycode.rename(columns={'English short name lower case':'Country', 'Alpha-2 code':'country code'}, inplace=True) 
#countrycode   #check to make sure the data looks how you want 

In [13]:
#Left outer join (called a merge in pandas). Adding c to dataframe name which stands for country code
topcities_c=pandas.merge(topcities, countrycode, on=['Country'], how='left')
#Find if any country codes are missing. This means the two countries' names didn't match up
#topcities_c[topcities_c['country code'].isna()] 

In [14]:
#Rename the countries that are mismatched (between the two ddataframes) so that they will match
topcities.Country = topcities.Country.replace({"US": "United States Of America","United States": "United States Of America",
                    "UAE": "United Arab Emirates", "Hong Kong China":"Hong Kong","UK":"United Kingdom",
                    "Macau China":"Macao", "South Korea": "Korea, Republic of (South Korea)"})
#Do the merge again and then do another check to see if you missed any other countries
topcities_c=pandas.merge(topcities, countrycode, on=['Country'], how='left')
#topcities_c[topcities_c['country code'].isna()]
#You can also re-rename country values if any are too long or another name is preferred
topcities_c.Country = topcities_c.Country.replace({"United States Of America": "United States",
                            "Macao":"Macau", "Korea, Republic of (South Korea)": "South Korea"})

STEP 3- ADD CITY POPULATIONS
Population dataset found here: https://www.kaggle.com/i2i2i2/cities-of-the-world

In [16]:
#Save it as a dataframe
population=pandas.read_csv('cities15000.csv',encoding = "ISO-8859-1")
#Get rid of columns we don't need
population= population.drop(['asciiname','geonameid','feature class','feature code','dem','elevation',
        'cc2','admin1 code', 'admin2 code','admin3 code','admin4 code','modification date'], axis=1)
#Rename "name" to "City" 
population.rename(columns={'name':'City'}, inplace=True)

In [17]:
#Merge dataframes
topcities_cp=pandas.merge(topcities_c, population, how='left', left_on=['City', 'country code'], right_on=['City', 'country code'])
#Check for null values in population column. This happens when the city and country names didn't match up
topcities_cp[topcities_cp['population'].isna()]

,Rank,City,Country,Arrivals 2016,Arrivals 2017,Arrivals 2018,Growth 2017,Growth 2018,Rank 2012-17,Rank 2017-18,country code,alternatenames,latitude,longitude,population,timezone
58,59,St. Petersburg,Russia,2840000.0,3600000.0,3996000.0,26.8%,11.0%,9,1,RU,NaN,NaN,NaN,NaN,NaN
82,83,Frankfurt,Germany,2406200.0,2496000.0,2582100.0,3.7%,3.4%,3,5,DE,NaN,NaN,NaN,NaN,NaN
96,97,Batam,Indonesia,1638600.0,2227500.0,2492600.0,35.9%,11.9%,31,6,ID,NaN,NaN,NaN,NaN,NaN


In [18]:
#Find those cities in the population dataset by looking in the alternatenames column
#See what the population dataset's main name for that city is and make note of it
population[population.alternatenames.str.contains('Frankfurt',na=False)]   #Frankfurt am Main
population[population.alternatenames.str.contains('St. Petersburg',na=False)]  #Saint Petersburg
population[population.alternatenames.str.contains('Batam',na=False)]  
#No city in Indonesia (only Cambodia) with that name so I will need to look for it manually

,City,alternatenames,latitude,longitude,country code,population,timezone
13121,Batam,"BBM,Bat Dambang,Batambang,Batambangas,Batdamba...",13.10271,103.19822,KH,150444,Asia/Phnom_Penh


In [19]:
#Change the names for those cities that were found
topcities_c.City = topcities_c.City.replace({"Frankfurt": "Frankfurt am Main",
                                                         "St. Petersburg":"Saint Petersburg"})
#Do the merge again and check for nulls again. Adding p to dataframe name for population
topcities_cp=pandas.merge(topcities_c, population, how='left', left_on=['City', 'country code'], right_on=['City', 'country code'])
topcities_cp[topcities_cp['population'].isna()]
#change back any cities names that the name was preferred
topcities_cp.City = topcities_cp.City.replace({"Frankfurt am Main":"Frankfurt"})

In [20]:
#Look for the missing Indonesian city
#If after looking it's still not there, look up the values and manually add them
#indonesiancities=population[population['country code'] == 'ID'] #created to only look at Indonesian cities
#indonesiancities.population= indonesiancities.population.astype(float) #change population to float
#indonesiancities.sort_values(by=['population'], ascending=False)  #sort by largest to smallest population
topcities_cp.at[96, 'population'] = 1236399
topcities_cp.at[96, 'latitude'] = "1.1301"
topcities_cp.at[96, 'longitude'] = "104.0529"
topcities_cp.at[96, 'timezone'] = "Asia/Jakarta"
topcities_cp.tail(10)   #check that it is there now

,Rank,City,Country,Arrivals 2016,Arrivals 2017,Arrivals 2018,Growth 2017,Growth 2018,Rank 2012-17,Rank 2017-18,country code,alternatenames,latitude,longitude,population,timezone
90,91,Lima,Peru,2151000.0,2348500.0,2583700.0,9.2%,10.0%,2,4,PE,"Gorad Lima,LIM,Lim,Lima,Limae,Limaq,Lime,Limma...",-12.04318,-77.02824,7737002,America/Lima
91,92,Nice,France,2164800.0,2292500.0,2331500.0,5.9%,1.7%,16,7,FR,"Gorad Nica,NCE,Nica,Nicaea,Nicc,Nicca,Niccae,N...",43.70313,7.26608,338620,Europe/Paris
92,93,Santiago,Chile,1933000.0,2275600.0,2648100.0,17.7%,16.4%,17,9,CL,"Ciles Santjagas,CiudadSantiago,SCL,Sanctiacobi...",-33.45694,-70.64827,4837295,America/Santiago
93,94,Rio de Janeiro,Brazil,2309000.0,2251300.0,2305300.0,-2.5%,2.4%,4,7,BR,"Ciutat de Rio de Janeiro,Gorad Rya-deh-Zhanehj...",-22.90278,-43.2075,6023699,America/Sao_Paulo
94,95,Abu Dhabi,United Arab Emirates,2054900.0,2244500.0,2402800.0,9.2%,7.1%,42,1,AE,"A-pu-that-pi,AEbu Saby,AUH,Aboe Dhabi,Abou Dab...",24.46667,54.36667,603492,Asia/Dubai
95,96,Porto,Portugal,1969300.0,2232500.0,2391500.0,13.4%,7.1%,42,1,PT,"OPO,Oporto,Porto,Portu,Portus Cale,bo tu,bwrtw...",41.14961,-8.61099,249633,Europe/Lisbon
96,97,Batam,Indonesia,1638600.0,2227500.0,2492600.0,35.9%,11.9%,31,6,ID,NaN,1.1301,104.0529,1236399,Asia/Jakarta
97,98,Doha,Qatar,2849000.0,2200000.0,2210000.0,-22.8%,0.5%,31,10,QA,"Ad Dawha,Ad Dawhah,Ad Dawü©a,Ad Dawü©ah,Ad D...",25.28545,51.53096,344939,Asia/Qatar
98,99,Rhodes,Greece,2091100.0,2178700.0,2281100.0,4.2%,4.7%,1,3,GR,"RHO,Rhodes,Rhodes Town,Rhodos,Rodas,Rodes,Rodh...",36.43556,28.22199,56128,Europe/Athens
99,100,Krabi,Thailand,2021700.0,2163200.0,2362100.0,7.0%,9.2%,9,2,TH,"Amphoe Muang Krabi,Amphoe Mueang Krabi,Amphoe ...",8.07257,98.91052,31219,Asia/Bangkok


STEP 4- ADD CITY SAFETY INDEX
https://ceoworld.biz/2019/08/01/the-worlds-safest-cities-ranking-2019/  
I will add this list to ask the question of what cities people are willing to travel to despite being unsafe
It was difficult to find a dataset with a lot of cities. This one has around 75% of the cities. Data is from 2019 (assumably from 2018).

In [22]:
#safety of cities
safetyindex= pandas.read_csv("safetyofcities.csv", sep=";")

In [23]:
#adding s to dataframe name for safety index
topcities_cps=pandas.merge(topcities_cp, safetyindex, how='left', left_on=['City', 'Country'], right_on=['City', 'Country'])

In [24]:
topcities_cps[topcities_cps['Safety Index'].isna()]
topcities_cp.City = topcities_cp.City.replace({"Ha Noi": "Hanoi","Penang City":"Penang",
                    "Kolkatta":"Kolkata","Buenos Aires City": "Buenos Aires"})
safetyindex.City=safetyindex.City.replace({"Krakow (Cracow)":"Krakow","Tel Aviv-Yafo":"Tel Aviv",
                                          "Ad Dammam":"Dammam City"})
topcities_cps=pandas.merge(topcities_cp, safetyindex, how='left', left_on=['City', 'Country'], right_on=['City', 'Country'])
#topcities_cps[topcities_cps['Safety Index'].isna()]

STEP 5- ADD COUNTRY HUMAN DEVELOPMENT INDEX
http://hdr.undp.org/sites/default/files/2018_human_development_statistical_update.pdf
https://en.wikipedia.org/wiki/List_of_countries_by_Human_Development_Index
Above are figures for 2017
https://www.ceicdata.com/en/indicator/macau/gdp-per-capita
https://www.bti-project.org/fileadmin/files/BTI/Downloads/Reports/2018/pdf/BTI_2018_Taiwan.pdf
https://www.bti-project.org/en/reports/country-reports/detail/itc/TWN/

In this dataset there were citations so I searched for a letter in between spaces " \w " and replaced with ""
There was one" \w,\w " as well

In [27]:
HDI=pandas.read_csv('HDI_country_rank.csv', sep=';')

In [28]:
#Get rid of columns we don't need
HDI.dtypes
HDI= HDI.drop(['HDI rank','GNI per capita rank minus HDI rank','HDI rank.1'], axis=1)

In [29]:
#merge with the main dataframe. Adding h to dataframe name for HDI
topcities_cpsh=pandas.merge(topcities_cps, HDI, how='left', on=['Country'])
topcities_cpsh[topcities_cpsh['Human Development Index'].isna()]
#Rename Country Czechia to Czech Republic
HDI.Country = HDI.Country.replace({"Czechia": "Czech Republic"})
#Merge and check again
topcities_cpsh=pandas.merge(topcities_cps, HDI, how='left', on=['Country'])
topcities_cpsh[topcities_cpsh['Human Development Index'].isna()]
#topcities_cpsh #manual check

,Rank,City,Country,Arrivals 2016,Arrivals 2017,Arrivals 2018,Growth 2017,Growth 2018,Rank 2012-17,Rank 2017-18,...,longitude,population,timezone,Safety Rank,Safety Index,Human Development Index,Life expectancy,Expected years of schooling,Mean years of schooling,Gross national income per capita


STEP 5- GET IMAGES
images are from gettyimages.com

In [32]:
#Get the list of cities in a list
citylist= topcities_cpsh["City"].tolist()
#citylist  #check to make sure the variable saved all the cities to a list

In [74]:
#loops thru all cities and add each to a getty images url to do a getty image search
#on that page, you use the css selector to return the html of the first image from the search
#you search for src from the html and save the urls into a new list

#old cssSelector (new cssSelector was made 10/21/2020)
#cssSelector= '''body > div.content_wrapper > section > div > main > section > div.search-content__gallery-pager-wrapper > div > div.search-content__gallery > div > div.search-content__gallery-assets > gix-asset:nth-child(1) > article > a > figure > img'''
cssSelector='''body > div.content_wrapper > section > div > main > section > div.search-content__gallery-pager-wrapper > div > div.search-content__gallery > div > div.search-content__gallery-assets > gi-asset > article > div > meta'''
imageurls= []

for i in range(len(citylist)):
    gettyurl='https://www.gettyimages.com/photos/'+citylist[i].replace(" ", "-")
    g= requests.get(gettyurl)
    p= bs4.BeautifulSoup(g.text, 'html.parser')
    n = p.select(cssSelector)
    f=re.compile(r'''"http(.+)" ''')
    a_string= "http"+f.findall(str(n))[0]
    split_string = a_string.split(" ", 1)
    imageurls.append(split_string[0])
    #f=re.compile(r'''src="(.+)"''')
    #imageurls.append(f.findall(str(n[0]))[0])

#gettyurls
#imageurls  #check that there is a list of image urls

In [77]:
#adding i for image and then adding each value to the column "City images" 
topcities_cpshi = topcities_cpsh
topcities_cpshi['City images']= imageurls

In [78]:
topcities_cpshi

,Rank,City,Country,Arrivals 2016,Arrivals 2017,Arrivals 2018,Growth 2017,Growth 2018,Rank 2012-17,Rank 2017-18,...,population,timezone,Safety Rank,Safety Index,Human Development Index,Life expectancy,Expected years of schooling,Mean years of schooling,Gross national income per capita,City images
0,1,Hong Kong,Hong Kong,26552700.0,27880300.0,29827200.0,5.0%,7.0%,0,0,...,7012738,Asia/Hong_Kong,10.0,82.00,0.933,84.10,16.3,12.0,58420,https://media.gettyimages.com/photos/cityscape...
1,2,Bangkok,Thailand,20698300.0,22453900.0,23688800.0,8.5%,5.5%,0,0,...,5104476,Asia/Bangkok,155.0,58.61,0.755,75.50,14.7,7.6,15516,https://media.gettyimages.com/photos/wat-arun-...
2,3,London,United Kingdom,19059500.0,19827800.0,20715900.0,4.0%,4.5%,0,0,...,7556900,Europe/London,236.0,47.76,0.922,81.70,17.4,12.9,39116,https://media.gettyimages.com/photos/the-big-b...
3,4,Singapore,Singapore,16604000.0,17618800.0,18551200.0,6.1%,5.3%,0,1,...,3547809,Asia/Singapore,62.0,71.64,0.932,83.20,16.2,11.5,82503,https://media.gettyimages.com/photos/gardens-b...
4,5,Macau,Macau,15704000.0,17337200.0,18931400.0,10.4%,9.2%,1,1,...,520400,Asia/Macau,NaN,NaN,0.909,83.99,12.4,12.0,77903,https://media.gettyimages.com/photos/macau-sky...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,Porto,Portugal,1969300.0,2232500.0,2391500.0,13.4%,7.1%,42,1,...,249633,Europe/Lisbon,135.0,60.75,0.847,81.40,16.3,9.2,27315,https://media.gettyimages.com/photos/porto-cit...
96,97,Batam,Indonesia,1638600.0,2227500.0,2492600.0,35.9%,11.9%,31,6,...,1236399,Asia/Jakarta,NaN,NaN,0.694,69.40,12.8,8.0,10846,https://media.gettyimages.com/photos/scenic-vi...
97,98,Doha,Qatar,2849000.0,2200000.0,2210000.0,-22.8%,0.5%,31,10,...,344939,Asia/Qatar,2.0,88.43,0.856,78.30,13.4,9.8,116818,https://media.gettyimages.com/photos/doha-skyl...
98,99,Rhodes,Greece,2091100.0,2178700.0,2281100.0,4.2%,4.7%,1,3,...,56128,Europe/Athens,NaN,NaN,0.870,81.40,17.3,10.8,24648,https://media.gettyimages.com/photos/rhodes-la...


In [ ]:
#Images of cities were retrieved except for Nice (France) due to name being not unique. 
#Next, create another image url column to get backup image URLS
#Concatenate the city and country into one list then run it again
countrylist= topcities_cpshi["Country"].tolist()

In [80]:
#2nd imageurl column- backup for more images
imageurls2=[]
#list(zip(citylist,countrylist))
for i in range(len(citylist)):
    #print((citylist[i]+'-'+countrylist[i]).replace(" ", "-"))
    gettyurl='https://www.gettyimages.com/photos/'+((citylist[i]+'-'+countrylist[i]).replace(" ", "-"))
    g= requests.get(gettyurl)
    p= bs4.BeautifulSoup(g.text, 'html.parser')
    n=p.select(cssSelector)
    f=re.compile(r'''"http(.+)" ''')
    a_string= "http"+f.findall(str(n))[0]
    split_string = a_string.split(" ", 1)
    imageurls2.append(split_string[0])
    #f=re.compile(r'''src="(.+)"''')
    #imageurls2.append(f.findall(str(n[0]))[0])

In [82]:
#adding 2 for 2 images and then adding each value to the column "City images2" 
topcities_cpshi2 = topcities_cpshi
topcities_cpshi2['City images2']= imageurls2
topcities_cpshi2

,Rank,City,Country,Arrivals 2016,Arrivals 2017,Arrivals 2018,Growth 2017,Growth 2018,Rank 2012-17,Rank 2017-18,...,timezone,Safety Rank,Safety Index,Human Development Index,Life expectancy,Expected years of schooling,Mean years of schooling,Gross national income per capita,City images,City images2
0,1,Hong Kong,Hong Kong,26552700.0,27880300.0,29827200.0,5.0%,7.0%,0,0,...,Asia/Hong_Kong,10.0,82.00,0.933,84.10,16.3,12.0,58420,https://media.gettyimages.com/photos/cityscape...,https://media.gettyimages.com/photos/cityscape...
1,2,Bangkok,Thailand,20698300.0,22453900.0,23688800.0,8.5%,5.5%,0,0,...,Asia/Bangkok,155.0,58.61,0.755,75.50,14.7,7.6,15516,https://media.gettyimages.com/photos/wat-arun-...,https://media.gettyimages.com/photos/wat-arun-...
2,3,London,United Kingdom,19059500.0,19827800.0,20715900.0,4.0%,4.5%,0,0,...,Europe/London,236.0,47.76,0.922,81.70,17.4,12.9,39116,https://media.gettyimages.com/photos/the-big-b...,https://media.gettyimages.com/photos/london-br...
3,4,Singapore,Singapore,16604000.0,17618800.0,18551200.0,6.1%,5.3%,0,1,...,Asia/Singapore,62.0,71.64,0.932,83.20,16.2,11.5,82503,https://media.gettyimages.com/photos/gardens-b...,https://media.gettyimages.com/photos/gardens-b...
4,5,Macau,Macau,15704000.0,17337200.0,18931400.0,10.4%,9.2%,1,1,...,Asia/Macau,NaN,NaN,0.909,83.99,12.4,12.0,77903,https://media.gettyimages.com/photos/macau-sky...,https://media.gettyimages.com/photos/macau-sky...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,Porto,Portugal,1969300.0,2232500.0,2391500.0,13.4%,7.1%,42,1,...,Europe/Lisbon,135.0,60.75,0.847,81.40,16.3,9.2,27315,https://media.gettyimages.com/photos/porto-cit...,https://media.gettyimages.com/photos/porto-cit...
96,97,Batam,Indonesia,1638600.0,2227500.0,2492600.0,35.9%,11.9%,31,6,...,Asia/Jakarta,NaN,NaN,0.694,69.40,12.8,8.0,10846,https://media.gettyimages.com/photos/scenic-vi...,https://media.gettyimages.com/photos/scenic-vi...
97,98,Doha,Qatar,2849000.0,2200000.0,2210000.0,-22.8%,0.5%,31,10,...,Asia/Qatar,2.0,88.43,0.856,78.30,13.4,9.8,116818,https://media.gettyimages.com/photos/doha-skyl...,https://media.gettyimages.com/photos/doha-skyl...
98,99,Rhodes,Greece,2091100.0,2178700.0,2281100.0,4.2%,4.7%,1,3,...,Europe/Athens,NaN,NaN,0.870,81.40,17.3,10.8,24648,https://media.gettyimages.com/photos/rhodes-la...,https://media.gettyimages.com/photos/beautiful...


STEP 6- CLEAN AND SAVE TO A CSV

In [84]:
#topcities_cpshi2.dtypes
#Rename some columns and remove alternatenames column
topcities_cpshi2.rename(columns={'country code':'Country Code','latitude':'Latitude','longitude':'Longitude',
                               'population':'Population','timezone':'Timezone'}, inplace=True)
#topcities_cpshi2= topcities_cpshi2.drop(['alternatenames','Safety Rank'], axis=1)

In [85]:
#check again for dups
try:
    pandas.concat(g for _, g in topcities_cpshi2.groupby("City") if len(g) > 1) 
except:
    print("There's no dups")

There's no dups


In [86]:
#Finished set! Save to a csv
topcities_cpshi2.to_csv(r'destinations.csv')